In [393]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [428]:
def green(img):
    shape = np.shape(img)
    return np.count_nonzero(img==255)/(shape[0]*shape[1])
def show_images(im,huesat_mask,save_outputs,filename):
    figwidth, figheight = plt.rcParams["figure.figsize"]
    fig, axes = plt.subplots(1, 2, figsize=(2 * figwidth, 1*figheight))
    
    axes[0].imshow(im)
    axes[1].imshow(huesat_mask)
    if save_outputs:
        fig.savefig(f'green_plots/{filename}.png')
        plt.close()

In [429]:
def one_img(filename,min_hue=40,max_hue=70,sat_min=0.2,min_val = 20,save_outputs=False,display=False):
    read_from = f'Cropped/{filename}.png'
    
    im = Image.open(read_from).convert('HSV')
    Hue = np.array(im.getchannel('H'))
    rgb = Image.open(read_from).convert('RGB')
    a = np.asarray(rgb, int)
    
    m = np.min(a,2).T
    M = np.max(a,2).T
    C = M - m
    Cmsk = C != 0
    V = M
    S = np.zeros_like(Hue, float).T
    S[Cmsk] = ((C[Cmsk]) / V[Cmsk])
    
    huesat_mask = np.zeros_like(Hue, dtype=np.uint8) 
    huesat_mask[(Hue>min_hue) & (Hue<max_hue) & (S.T > sat_min)& (V.T > min_val)] = 255
    
    if display or save_outputs:
        show_images(im,huesat_mask,save_outputs,filename)
    
    return green(huesat_mask)
    
def graph_it_all(filename,min_hue=40,max_hue=70,sat_min=0.2,min_val=20,save_outputs=False,test_mode=False,display_col=0):
    arr = []
    r = 8
    c = 6
    for x in range(r):
        tmp = []
        for y in range(c):
            if x==display_col and test_mode == True:
                tmp.append(one_img(f'{filename}_{x}_{y}',min_hue,max_hue,sat_min,min_val,save_outputs,display=True))
            else:
                tmp.append(one_img(f'{filename}_{x}_{y}',min_hue,max_hue,sat_min,min_val,save_outputs,display=False))
        arr.append(tmp)
    return arr

In [430]:
test = graph_it_all("screenshot",save_outputs=True,test_mode=True,display_col=0)

In [367]:
np.savetxt("GreenPercentagesBoston/grid_scores.csv", test, delimiter=",")